<a href="https://colab.research.google.com/github/Bharat67889/autoquotes/blob/main/instvidgenrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Auto start
from IPython.display import Javascript
Javascript('google.colab.kernel.execute("run_all()")')



# ---------------- SETUP ----------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import json
import os, random, time, sys
from moviepy.editor import ImageClip, AudioFileClip
from PIL import Image, ImageDraw, ImageFont
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# ---------------- INSTALL DEPENDENCIES ----------------
!apt-get update -qq
!apt-get install -y ffmpeg fonts-dejavu
!pip install moviepy gspread oauth2client pillow

# ---------------- CONFIG ------------------
SERVICE_ACCOUNT_JSON = "/content/drive/MyDrive/ColabData/crdidentals.json"  # ✅ use full Drive path
SHEET_ID = "1-_wok2tVaNlU2MMZd7IGRjtEBVyh4zNVNjMZLXN0mG0"
SHEET_NAME = "Quotes"
DRIVE_FOLDER = "/content/drive/MyDrive/AutoQuotes"
MUSIC_FOLDER = "/content/drive/MyDrive/AutoQuotesMusic"  # 🎵 your music folder
VIDEO_WIDTH = 1080
VIDEO_HEIGHT = 1920
VIDEO_DURATION = 10
FPS = 30
FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
MARGIN = 120  # horizontal padding from left and right
# ------------------------------------------

# ---------------- HELPER FUNCTIONS ----------------
def log(msg):
    print(msg, flush=True)

# Mount Google Drive (just once)
log("🔹 Mounting Google Drive...")
os.makedirs(DRIVE_FOLDER, exist_ok=True)
os.makedirs(MUSIC_FOLDER, exist_ok=True)
log("✅ Drive mounted and folders ready.")

# Connect to Google Sheets
log("🔹 Connecting to Google Sheets...")
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_JSON, scope)
client = gspread.authorize(creds)
sheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)
rows = sheet.get_all_records()
log(f"✅ Loaded {len(rows)} rows from sheet '{SHEET_NAME}'.")

# ---------------- GENERATE VIDEOS ----------------
for idx, row in enumerate(rows):
    try:
        log(f"\n🔹 Processing row {idx+1} / {len(rows)}")
        quote = row['Quote']
        log(f"Quote: {quote}")

        # 1️⃣ Create an image with PIL
        img = Image.new('RGB', (VIDEO_WIDTH, VIDEO_HEIGHT), color=(0, 0, 0))
        draw = ImageDraw.Draw(img)
        font_size = 45
        font = ImageFont.truetype(FONT_PATH, font_size)

        # Wrap text manually
        lines = []
        words = quote.split()
        line = ""
        for word in words:
            test_line = line + " " + word if line else word
            bbox = draw.textbbox((0, 0), test_line, font=font)
            line_width = bbox[2] - bbox[0]
            if line_width < VIDEO_WIDTH - 2 * MARGIN:
                line = test_line
            else:
                lines.append(line)
                line = word
        if line:
            lines.append(line)

        # Draw text slightly above center
        total_text_height = len(lines) * font_size
        y_text = (VIDEO_HEIGHT - total_text_height) // 2 - 190
        for line in lines:
            bbox = draw.textbbox((0, 0), line, font=font)
            line_width = bbox[2] - bbox[0]
            x_text = MARGIN + (VIDEO_WIDTH - 2 * MARGIN - line_width) // 2
            draw.text((x_text, y_text), line, font=font, fill=(255, 255, 255))
            y_text += font_size

        # Save temp image
        timestamp = int(time.time())
        img_path = os.path.join("/tmp", f"quote_{timestamp}.png")
        img.save(img_path)
        log(f"✅ Image created: {img_path}")

        # 2️⃣ Convert image to video
        clip = ImageClip(img_path).set_duration(VIDEO_DURATION)

        # 3️⃣ Add random background music
        music_files = [os.path.join(MUSIC_FOLDER, f) for f in os.listdir(MUSIC_FOLDER) if f.endswith((".mp3", ".wav", ".m4a"))]
        if music_files:
            music_file = random.choice(music_files)
            audio_clip = AudioFileClip(music_file)
            if audio_clip.duration > VIDEO_DURATION:
                audio = audio_clip.subclip(0, VIDEO_DURATION)
            else:
                audio = audio_clip.set_duration(VIDEO_DURATION)
            clip = clip.set_audio(audio)
            log(f"✅ Music added: {music_file}")
        else:
            log("⚠️ No music files found in folder, skipping background music")

        # 4️⃣ Save final video
        safe_name = "".join([c if c.isalnum() else "_" for c in quote[:20]])
        video_file = os.path.join(DRIVE_FOLDER, f"{safe_name}_{timestamp}.mp4")
        log(f"🔹 Rendering video: {video_file}")
        clip.write_videofile(video_file, fps=FPS, audio_codec="aac", logger="bar")
        log(f"✅ Video created successfully: {video_file}")

    except Exception as e:
        log(f"❌ Error processing row {idx+1}: {e}")


Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra
The following NEW packages will be installed:
  fonts-dejavu fonts-dejavu-core fonts-dejavu-extra
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 3,085 kB of archives.
After this operation, 10.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-extra all 2.37-2build1 [2,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-dejavu all 2.37-2buil

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabData/crdidentals.json'